In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import Bounds
from calibration.solver_calibration import Calibration
from models.jump_vasicek import JumpVasicek
from models.jump_cir import JumpCIR
from models.vasicek import Vasicek

In [2]:
inital_model_params = {
    "kappa": 0.5,
    "mu_r": 0.03,
    "sigma": 0.03,
    "mu": 0,
    "gamma": 0.01,
    "h": 10
}

In [3]:
yield_curve_full = pd.read_csv("data/yield_curve02132021.csv")
yield_curve = pd.DataFrame(yield_curve_full.iloc[0, 1:]/100).astype(float)
yield_curve["Maturity"] = list(range(1, len(yield_curve) + 1))
yield_curve.columns = ["Yield", "Maturity"]
yield_curve["Cum. Yield"] = yield_curve["Yield"] * yield_curve["Maturity"]
yield_curve["Price"] = np.exp(-yield_curve["Cum. Yield"])

In [4]:
yield_curve

,Yield,Maturity,Cum. Yield,Price
SVENY01,0.000773,1,0.000773,0.999227
SVENY02,0.001020,2,0.002040,0.997962
SVENY03,0.002010,3,0.006030,0.993988
SVENY04,0.003401,4,0.013604,0.986488
SVENY05,0.004972,5,0.024860,0.975446
SVENY06,0.006577,6,0.039462,0.961306
SVENY07,0.008130,7,0.056910,0.944679
SVENY08,0.009580,8,0.076640,0.926223
SVENY09,0.010904,9,0.098136,0.906526
SVENY10,0.012091,10,0.120910,0.886114


In [5]:
prices = yield_curve["Price"].values
Ts = yield_curve["Maturity"].values

jump_vasicek = JumpVasicek(inital_model_params)
optimize_args = ("kappa", "mu_r", "sigma", "gamma")
bounds = Bounds([0, 0, 0.01, 0.01], [1, 0.2, 0.2, 0.1])


In [26]:
if __name__ == "__main__":
    calibrator = Calibration(
        model_class=JumpVasicek, n=100, m=104, r0=0.001,
        model_params=inital_model_params, optimize_args=optimize_args)
    optimal_jv = calibrator.calibrate(
        initial_values=(inital_model_params["kappa"], inital_model_params["mu_r"], inital_model_params["sigma"], inital_model_params["gamma"]),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )

In [28]:
optimal_jv

      fun: 0.281366176445085
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-8.33688174e-02,  5.82645043e-05, -8.03801470e-06,  6.60533117e-01])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 305
      nit: 46
   status: 0
  success: True
        x: array([1.        , 0.02748451, 0.1615514 , 0.001     ])

In [22]:
if __name__ == "__main__":
    calibrator = Calibration(
        model_class=Vasicek, n=100, m=104, r0=0.001,
        model_params=inital_model_params, optimize_args=optimize_args)
    optimal_v = calibrator.calibrate(
        initial_values=(inital_model_params["kappa"], inital_model_params["mu_r"], inital_model_params["sigma"]),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )

In [25]:
optimal_v

      fun: 0.2807072862154902
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-8.39214087e-02, -2.68512990e-04,  3.66540132e-05,  0.00000000e+00])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 270
      nit: 40
   status: 0
  success: True
        x: array([1.        , 0.02816315, 0.16698004, 0.01      ])

In [33]:
if __name__ == "__main__":
    bounds = Bounds([0, 0, 0.01, 0.01], [1, 0.2, 0.2, 0.1])
    calibrator = Calibration(
        model_class=JumpVasicek, n=100, m=52, r0=0.001,
        model_params=inital_model_params, optimize_args=optimize_args)
    optimal_jv = calibrator.calibrate(
        initial_values=(inital_model_params["kappa"], inital_model_params["mu_r"], inital_model_params["sigma"], inital_model_params["gamma"]),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )

In [17]:
if __name__ == "__main__":
    bounds = Bounds([0, 0, 0.01, 0.01], [1, 0.2, 0.2, 0.1])
    calibrator = Calibration(
        model_class=JumpCIR, n=100, m=52, r0=0.001,
        model_params=inital_model_params, optimize_args=optimize_args)
    optimal_jcir = calibrator.calibrate(
        initial_values=(inital_model_params["kappa"], inital_model_params["mu_r"], inital_model_params["sigma"], inital_model_params["gamma"]),
        Ts=Ts,
        prices=prices,
        bounds=bounds
    )

KeyboardInterrupt: 

In [35]:
optimal_jv

      fun: 0.28098401744523255
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-7.71780417e-02, -2.60347299e-06,  3.64483727e-01,  1.14908083e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 210
      nit: 28
   status: 0
  success: True
        x: array([1.        , 0.02352761, 0.01      , 0.03249692])

In [ ]:
optimal_jcir

In [7]:
rj, dt, nj, Pj, time_step, stoch_step = 0.0008159829113362281, 0.019230769230769232, -1.4917360548726768, 2.0, -1.1418600080995072e-05, -0.0009986983396235035

In [8]:
Pj

2.0

In [9]:
lower_bound = (-rj - time_step - stoch_step)/Pj
upper_bound = -lower_bound

In [12]:
from scipy.stats import truncnorm
Jj = truncnorm(lower_bound, upper_bound, loc=0, scale=0.01).rvs(1)[0]

ValueError: Domain error in arguments.

In [14]:
upper_bound

-9.706701418413525e-05

In [15]:
lower_bound

9.706701418413525e-05

In [16]:
rj

0.0008159829113362281